In [1]:
import numpy as np
import subprocess
import pickle
import time
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.lammpsdata import write_lammps_data, read_lammps_data
import sys
from tqdm import tqdm
from scipy.constants import physical_constants
kB = physical_constants["Boltzmann constant in eV/K"][0]

In [2]:
# Load saved arrays for testing
T=1073
swaps = np.load("MC_test_traj/1/swap_atoms_all_steps.npy")
rands = np.load("MC_test_traj/1/rands_all_steps.npy")
en_stored = np.load("MC_test_traj/1/Eng_all_steps.npy")
accepts = np.load("MC_test_traj/1/accepts_all_steps.npy")
elems = ["Co", "Ni", "Cr", "Fe", "Mn"]

In [3]:
en_stored.shape, accepts.shape

((501,), (500,))

## First make the test lammps command file

In [4]:
with open("MC_test_traj/1/in_run_1.minim", "r") as fl:
    commandList_run_1 = fl.readlines()

with open("MC_test_traj/1/in.minim", "r") as fl:
    commandList_run_2 = fl.readlines()

# check that the commands were the same
assert commandList_run_1 == commandList_run_2

# Let's view the commands for the test command file
for command in commandList_run_1:
    print(command)

units 	 metal

atom_style 	 atomic

atom_modify 	 map array

boundary 	 p p p

atom_modify 	 sort 0 0.0

read_data 	 inp_MC.data

pair_style 	 meam

pair_coeff 	 * * /mnt/WorkPartition/Work/Research/UIUC/MDMC/Utils/pot/library.meam Co Ni Cr Fe Mn /mnt/WorkPartition/Work/Research/UIUC/MDMC/Utils/pot/params.meam Co Ni Cr Fe Mn

minimize		 1e-08 0.0 500 1000000

variable x equal pe

print "$x" file Eng.txt


In [5]:
# Change the following potential file location line since we're one directory up 
with open("in_test.minim", "w") as fl:
    fl.writelines(commandList_run_1)

## Check the energies of all the supercells

In [6]:
with open("MC_test_traj/1/superInitial.pkl", "rb") as fl:
    sup0 = pickle.load(fl)
len(sup0)

499

In [8]:
NEqb = 5 # we had 5 thermalization steps

sup_run = sup0.copy() # this will be the running supercell which we'll compare with others

supercells = []

# Compute energy
write_lammps_data("inp_MC.data", sup0, specorder=elems)
cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
rt = cmd.wait()
assert rt == 0

with open("inp_MC.data", "r") as fl:
    lammps_file_2 = fl.readlines()

# Check energy
with open("Eng.txt", "r") as fl:
    en = float(fl.readline().split()[0])
assert en_stored[0] == en 

# First do the swaps upto thermalization
for i in tqdm(range(NEqb), position=0, leave=True, ncols=65):
    acc = accepts[i]
    if acc == 1:
        # get the swap
        sw = swaps[i]
        site1 = sw[0]
        site2 = sw[1]
        
        # swap occupancies in the running supercell
        s1 = sup_run[site1].symbol
        s2 = sup_run[site2].symbol
        
        # check the swapping in the code
        tmp = sup_run[site1].symbol
        sup_run[site1].symbol = sup_run[site2].symbol
        assert tmp != sup_run[site1].symbol
        sup_run[site2].symbol = tmp
        assert sup_run[site2].symbol != sup_run[site1].symbol
        assert sup_run[site2].symbol == s1
        assert sup_run[site1].symbol == s2
    
    # Do the energy computation
    write_lammps_data("inp_MC.data", sup_run, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0
    with open("Eng.txt", "r") as fl:
        en = float(fl.readline().split()[0])
    
    # Check energy
    assert en == en_stored[i + 1], "{} {}".format(en, en_stored[i + 1])

with open("MC_test_traj/1/chkpt/supercell_{}.pkl".format(NEqb), "rb") as fl:
    sup1 = pickle.load(fl)
        
assert sup1 == sup_run
supercells.append(sup1)

# Up until here, the NEqb^th supercell (starting from 0) has been reached

# Now go through the rest of the supercells
for i in tqdm(range(NEqb + 1, en_stored.shape[0]), position=0, leave=True, ncols=65):
    # Compute energy
    with open("MC_test_traj/1/chkpt/supercell_{}.pkl".format(i), "rb") as fl:
        sup1 = pickle.load(fl)
    supercells.append(sup1)
    
    # compare occupancies
    # first check the if the move was accepted
    # accepts[i-1] is the decision to go from (i-1)th state to (i)th state
    acc = accepts[i-1]
    if acc == 0:
        assert sup1 == sup_run
    
    else:
        # get the swap
        sw = swaps[i-1]
        site1 = sw[0]
        site2 = sw[1]
        
        # swap occupancies in the running supercell
        tmp = sup_run[site1].symbol
        sup_run[site1].symbol = sup_run[site2].symbol
        assert tmp != sup_run[site1].symbol
        sup_run[site2].symbol = tmp
        assert sup_run[site2].symbol != sup_run[site1].symbol
        
        assert sup1 == sup_run, "{}".format(i) # Check that the correct supercell was stored.
    
    # compute energies with the running supercell
    write_lammps_data("inp_MC.data", sup_run, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0
    with open("Eng.txt", "r") as fl:
        en = float(fl.readline().split()[0])
    
    # Check energy
    assert en == en_stored[i], "{} {}".format(en, en_stored[i])
    
print("Energy assertions passed")

100%|██████████████████████████| 495/495 [11:25<00:00,  1.39s/it]

Energy assertions passed


## Match the rejected and accepted moves against the random numbers

In [9]:
rejected = []
accepted = []
for enInd in range(en_stored.shape[0] - 1):
    # If the exact same energy occurs, then move must have been rejected
    # Assuming there will be defnitely some change, even if small when a state is changed 
    if en_stored[enInd + 1] - en_stored[enInd] == 0.0:
        assert accepts[enInd] == 0
        rejected.append(enInd)
    else:
        assert accepts[enInd] == 1
        accepted.append(enInd)

In [10]:
# Now Let's check the swaps that were accepted
for move in tqdm(accepted, position=0, leave=True, ncols=65):
    if move < NEqb:
        # the "move"^th move takes us from the "move"^th to the "move + 1"^th state
        # we only have supercell saved from NEqb onwards - so we can check starting with
        # the one before that.
        continue
        
    sup_temp = supercells[move - NEqb].copy()
    tmp = sup_temp[swaps[move, 0]].symbol
    sup_temp[swaps[move, 0]].symbol = sup_temp[swaps[move, 1]].symbol
    sup_temp[swaps[move, 1]].symbol = tmp
    
    # check supercell
    assert sup_temp == supercells[move - NEqb + 1]
    
    # compute energy
    write_lammps_data("inp_MC.data", sup_temp, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()

    # Read energy
    with open("Eng.txt", "r") as fl:
        en_temp = float(fl.readline().split()[0])
    
    assert en_temp == en_stored[move + 1]
    de = en_stored[move + 1] - en_stored[move]

    test_num = np.exp(-de/(kB*T))
    rand = rands[move]

    assert rand < test_num

print("Acceptance tests okay.")

100%|██████████████████████████| 320/320 [07:17<00:00,  1.37s/it]

Acceptance tests okay.


In [11]:
# Let's first check a swap that was rejected
for move in tqdm(rejected, position=0, leave=True, ncols=65): # index of initial state of the move
    assert np.allclose(en_stored[move], en_stored[move + 1])
    
    if move < NEqb: # the "move"^th move takes us from the "move"^th to the "move + 1"^th state
        continue # we only have supercell saved from NEqb onwards
        
    sup_temp = supercells[move - NEqb].copy()
    assert sup_temp == supercells[move - NEqb + 1]
    
    tmp = sup_temp[swaps[move, 0]].symbol
    sup_temp[swaps[move, 0]].symbol = sup_temp[swaps[move, 1]].symbol
    sup_temp[swaps[move, 1]].symbol = tmp

    # compute energy
    write_lammps_data("inp_MC.data", sup_temp, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0

    # Read energy
    with open("Eng.txt", "r") as fl:
        en_temp = float(fl.readline().split()[0])

    # Check what the random number was
    rand = rands[move]
    de = en_temp - en_stored[move]
    test_num = np.exp(-de/(kB*T))
    # Check that rand is greater than relative prob.
    assert rand >= test_num, "{} {}".format(rand, test_num)

print("Rejection checks okay.")

100%|██████████████████████████| 180/180 [04:00<00:00,  1.34s/it]

Rejection checks okay.
